---
# Initialisation




In [ ]:
!pip install eli5
!pip install -qq -U gensim
!pip install pyLDAvis
!pip install --upgrade numpy

In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import altair as alt #viz
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline # Pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer # Transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression # Logit model
from sklearn.metrics import classification_report # That's self explanatory
import eli5
from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # We'll use the faster multicore version of LDA
import plotly.graph_objs as go
import json


# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import spacy # Spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') # Instantiating English module

# Load the stop words list
nltk.download('stopwords')

nltk.download('punkt')




---


# Data Handling

In [98]:
data = pd.read_csv('/content/drive/MyDrive/Kickstarter.csv') # Copy the directory of the files on your computer here

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning:

Columns (14,18,20,24) have mixed types.Specify dtype option on import or set low_memory=False.



In [99]:
data

,backers_count,blurb,category,converted_pledged_amount,country,country_displayable_name,created_at,creator,currency,currency_symbol,...,source_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,urls,usd_exchange_rate,usd_pledged,usd_type
0,393,"Finally, studying is fun. Medcomic's combinati...","{""id"":253,""name"":""Webcomics"",""analytics_name"":...",20366,US,the United States,1438134995,"{""id"":1881653671,""name"":""Jorge Muniz"",""is_regi...",USD,$,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1446299928,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,20366.000000,international
1,209,Printing volume 2 of the epic fantasy graphic ...,"{""id"":253,""name"":""Webcomics"",""analytics_name"":...",7628,CA,Canada,1442258012,"{""id"":559623833,""name"":""Ally Rom Colthoff"",""is...",CAD,$,...,https://www.kickstarter.com/discover/categorie...,True,True,successful,1445697427,0.749857,"{""web"":{""project"":""https://www.kickstarter.com...",0.760194,7524.818407,international
2,327,"The sixth Weregeek comics compilation, a graph...","{""id"":253,""name"":""Webcomics"",""analytics_name"":...",17902,US,the United States,1436387555,"{""id"":749583352,""name"":""Alina Pete"",""slug"":""we...",USD,$,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1445542357,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,17902.000000,international
3,776,The second volume of the on-going space advent...,"{""id"":253,""name"":""Webcomics"",""analytics_name"":...",34781,US,the United States,1441231968,"{""id"":496629008,""name"":""Garth Graham"",""is_regi...",USD,$,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1445400039,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,34781.000000,international
4,148,Bar Dog is a humorous webcomic about a dog who...,"{""id"":253,""name"":""Webcomics"",""analytics_name"":...",10282,US,the United States,1380835236,"{""id"":64544729,""name"":""Carol Holsinger"",""is_re...",USD,$,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1445003944,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,10282.000000,international
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30108,16,Cthulhu Too: The Stranger in Yellow continues ...,"{""id"":285,""name"":""Plays"",""analytics_name"":""Pla...",1731,US,the United States,1494726965,"{""id"":1357253983,""name"":""Edward E Wise II"",""is...",USD,$,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1499223302,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,1731.000000,international
30109,33,I am bringing my solo show to Scotland!,"{""id"":285,""name"":""Plays"",""analytics_name"":""Pla...",3620,US,the United States,1495034667,"{""id"":253010467,""name"":""Elan Zafir"",""is_regist...",USD,$,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1499205600,1.000000,"{""web"":{""project"":""https://www.kickstarter.com...",1.000000,3620.000000,international
30110,28,An independent production of Mauritius by Ther...,"{""id"":285,""name"":""Plays"",""analytics_name"":""Pla...",5037,AU,Australia,1495424326,"{""id"":367454030,""name"":""Richard Cornally"",""is_...",AUD,$,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1499182258,0.744100,"{""web"":{""project"":""https://www.kickstarter.com...",0.767853,4881.297443,international
30111,8,Follow Benjamin and the voices in his head to ...,"{""id"":285,""name"":""Plays"",""analytics_name"":""Pla...",1313,GB,the United Kingdom,1492955270,"{""id"":1030415124,""name"":""Moss Banks Production...",GBP,£,...,https://www.kickstarter.com/discover/categorie...,True,False,successful,1501430407,1.282135,"{""web"":{""project"":""https://www.kickstarter.com...",1.313543,1282.135110,international




---


# Descriptive Analysis

In [100]:
# We want to know what are the differents categories and their subcategories

# Initialize an empty dictionary to store categories counts
category_count = {}

# Loop over each row in the dataset
for i in range(30113):
    # Extract the category and subcategory information from the "category" column
    category = json.loads(data["category"][i])
    name = category['name']
    
    # If the "parent_name" key is not present in the category dictionary, skip to the next row
    if "parent_name" not in category:
        continue
    
    parent_name = category['parent_name']
    
    # If the parent category is not already in the dictionary, add it
    if parent_name not in category_count:
        category_count[parent_name] = {}
    
    # Increment the count of the subcategory for the parent category
    if name not in category_count[parent_name]:
        category_count[parent_name][name] = 0
    category_count[parent_name][name] += 1

# Loop over the dictionary and print the frequency counts for each category and subcategory
for parent_name, counts in category_count.items():
    print(f"{parent_name}:")
    for name, count in counts.items():
        print(f"  {name}: {count}")

Comics:
  Webcomics: 1263
  Graphic Novels: 2400
Film & Video:
  Comedy: 2400
Music:
  R&B: 675
  Punk: 458
  Pop: 2364
Journalism:
  Audio: 966
  Video: 182
  Web: 610
  Print: 460
  Photo: 303
Food:
  Cookbooks: 937
  Community Gardens: 381
  Bacon: 223
  Drinks: 469
  Small Batch: 335
  Restaurants: 320
  Vegan: 190
  Farms: 131
  Farmer's Markets: 25
  Spaces: 82
  Food Trucks: 98
  Events: 840
Technology:
  Sound: 1101
  Software: 2400
Design:
  Graphic Design: 1273
  Product Design: 1828
  Civic Design: 6
  Toys: 147
  Typography: 6
  Interactive Design: 11
  Architecture: 16
Publishing:
  Literary Journals: 439
  Letterpress: 98
  Fiction: 2400
  Poetry: 12
  Literary Spaces: 194
Theater:
  Plays: 1822
  Spaces: 266
Art:
  Ceramics: 507


In [101]:
# Let's visualize it in a prettier way (You can pass hover the pie to see the subcategories inside)

# Collect the data
labels = []
values = []
hover_texts = []

for parent_name, counts in category_count.items():
    name_list = []
    for name, count in counts.items():
        name_list.append(f"{name}: {count}")
        values.append(count)
    labels.append(parent_name)
    hover_texts.append("<br>".join(name_list))

# Create the plot
fig = go.Figure(data=[go.Pie(labels=labels, values=values, hovertext=hover_texts, hoverinfo="label+text")])
fig.show()




---


# Setup of the Model



In [70]:
# Select only the columns 'blurb' and 'state'
data = data[['blurb', 'state']]

# Convert all values in the DataFrame to strings
data = data.astype(str)

In [71]:
data

,blurb,state
0,"Finally, studying is fun. Medcomic's combinati...",successful
1,Printing volume 2 of the epic fantasy graphic ...,successful
2,"The sixth Weregeek comics compilation, a graph...",successful
3,The second volume of the on-going space advent...,successful
4,Bar Dog is a humorous webcomic about a dog who...,successful
...,...,...
30108,Cthulhu Too: The Stranger in Yellow continues ...,successful
30109,I am bringing my solo show to Scotland!,successful
30110,An independent production of Mauritius by Ther...,successful
30111,Follow Benjamin and the voices in his head to ...,successful


In [73]:
# Clean the 'blurb' column by removing all non-word characters and digits, and converting to lowercase
data['blurb_clean'] = data['blurb'].apply(lambda x: re.sub(r'[^\w\s]|\d+', '', x.lower().strip()))

def remove_stop_words(data, column_name):
    # Define stop words for multiple languages
    stop_words_en = set(stopwords.words('english'))
    stop_words_fr = set(stopwords.words('french'))
    stop_words_sp = set(stopwords.words('spanish'))
    stop_words_ger = set(stopwords.words('german'))

    # Combine stop words from different languages into a single set
    stop_words = stop_words_en.union(stop_words_fr).union(stop_words_sp).union(stop_words_ger)
    
    # Remove stop words from the 'blurb_clean' column
    data[column_name] = data[column_name].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
    
    return data

data = remove_stop_words(data, 'blurb_clean')

In [75]:
# Count the number of projects in each unique value of the 'state' column
data["state"].value_counts()

successful    19468
failed         9176
canceled        931
live            538
Name: state, dtype: int64

In [76]:
# Change the "canceled" projects to "failed"
data.loc[data["state"] == "canceled", "state"] = "failed"

# Filter the dataset to only include "successful" and "failed" projects
data = data[data['state'].isin(['successful', 'failed'])]

In [ ]:
data['blurb_clean'] = data['blurb_clean'].apply(lambda x: ' '.join(word_tokenize(x)))

In [78]:
data_df = pd.DataFrame({'label':data['state'], 'text':data['blurb_clean']})

In [79]:
alt.Chart(data_df.label.value_counts().reset_index()).mark_bar(filled=True).encode(
    alt.X('label:Q', title='N Projects'),
    alt.Y('index:N', title='State')
)

alt.Chart(...)


*   There's around 10000 failed project and almost twice more of successful ones


In [80]:
# Equilibrate the number of input from the two states by selecting them randomly
rus = RandomUnderSampler(random_state=42)
data_df_res, y_res = rus.fit_resample(data_df, data_df['label'])

In [81]:
# Split the resampled data and target variable into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data_df_res['text'], y_res, test_size = 0.4, random_state = 42)
# We set the test set on 40% of the data

In [82]:
# Instantiate models and "bundle up as pipeline"
tfidf = TfidfVectorizer()
cls = LogisticRegression()

pipe = make_pipeline(tfidf, cls)

In [83]:
pipe.fit(X_train,y_train) # Fit model

Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('logisticregression', LogisticRegression())])

In [84]:
# Evaluate model performance on training set
y_eval = pipe.predict(X_train)
report = classification_report(y_train, y_eval)
print(report)

              precision    recall  f1-score   support

      failed       0.86      0.88      0.87      6049
  successful       0.88      0.86      0.87      6079

    accuracy                           0.87     12128
   macro avg       0.87      0.87      0.87     12128
weighted avg       0.87      0.87      0.87     12128





---


# Try Yourself !

In [85]:
def predict_sentiment(text):
    # Define the stop words for each language
    stop_words_en = set(stopwords.words('english'))
    stop_words_fr = set(stopwords.words('french'))
    stop_words_sp = set(stopwords.words('spanish'))
    stop_words_ger = set(stopwords.words('german'))
    # Combine the stop words for all languages
    stop_words = stop_words_en.union(stop_words_fr).union(stop_words_sp).union(stop_words_ger)

    # Define a function to preprocess the text before making predictions
    def preprocess_text(text):
        # Convert the text to a Pandas series
        text = pd.Series(text)
        # Remove special characters, digits, and convert to lowercase
        text = text.apply(lambda x: re.sub(r'[^\w\s]|\d+', '', x.lower().strip()))
        # Remove stop words
        text = text.apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))
        # Tokenize the text
        text = text.apply(lambda x: ' '.join(word_tokenize(x)))
        return text
    
    # Preprocess the input text
    t1_p = preprocess_text(text)

    # Make a prediction using the pre-trained pipeline
    prediction = pipe.predict(t1_p)
    # Generate an explanation for the prediction using ELI5
    explanation = display(eli5.show_prediction(pipe[1], t1_p[0], vec=pipe[0], target_names=['failed','successful']))

    return prediction, explanation

Enter a example to see how good or bad is your project description.

> Write in the code below after **`text = `**




In [88]:
text = "A sport innovation that will bring you more comfort."

prediction, explanation = predict_sentiment(text)
print("Prediction:", prediction)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



Prediction: ['successful']




---
# Now let's see a more global analysis

---
How the model clusterise the words and which words are positive or negative




In [89]:
# Initialize an empty list to store the tokenized texts
tokens = []

# Loop through the 'blurb_clean' column of the DataFrame using spaCy's nlp.pipe() function
for text in nlp.pipe(data['blurb_clean'], disable=["ner"]):
    
    # Use spaCy to extract the lemmatized lowercased tokens for each text that are either nouns, proper nouns, adjectives, or adverbs and not stop words or punctuations
    proj_tok = [token.lemma_.lower() for token in text 
                if token.pos_ in ['NOUN', 'PROPN', 'ADJ', 'ADV'] 
                and not token.is_stop
                and not token.is_punct] 
    
    # Append the list of tokens to the tokens list
    tokens.append(proj_tok)

In [ ]:
data['tokens'] = tokens

In [91]:
data_success = data[data['state'] == "successful"]

In [92]:
# Create a Dictionary from the articles: dictionary
dictionary = Dictionary(data_success['tokens'])
# filter out low-frequency / high-frequency stuff, also limit the vocabulary to max 1000 words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=1000)
# construct corpus using this dictionary
corpus = [dictionary.doc2bow(doc) for doc in data_success['tokens']]

In [93]:
# Training the model
lda_model = LdaMulticore(corpus, id2word=dictionary, num_topics=4, workers = 4, passes=10)

In [ ]:
# Let's try to visualize
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

In [95]:
 # Let's Visualize the differents clusters regrouping words from the same field together
pyLDAvis.display(lda_display)

In [96]:
# Display the top weighted features for each project's state
eli5.show_weights(pipe, top=20, target_names=['failed','successful'])

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



Weight?,Feature
+5.416,novel
+2.648,design
+2.515,graphic
+2.467,fantasy
+2.453,poster
+2.367,collection
+2.188,enamel
+2.115,fiction
+1.990,queer
+1.945,coffee
